In [18]:
from symmer.chemistry import MoleculeBuilder,xyz_from_pubchem
from symmer.symplectic import QuantumState
import numpy as np
from openfermion import FermionOperator

In [2]:
basis='sto-3g'
charge=0
geometry = xyz_from_pubchem('H2O')
# geometry ='3\n \nO\t0\t0\t0\nH\t0.2774\t0.8929\t0.2544\nH\t0.6068\t-0.2383\t-0.7169\n'

molecule = MoleculeBuilder(geometry=geometry,
                           charge=charge, 
                           basis=basis,
                           spin=0, 
                           run_fci=True, 
                           print_info=True,
                          symmetry=True)

Molecule geometry:
O	0	0	0
H	0.2774	0.8929	0.2544
H	0.6068	-0.2383	-0.7169


CISD converged?  True
FCI converged?  True
FCI converged?  True

HF energy:   -74.96444758277003
MP2 energy:  -75.00099822744205
CCSD energy: -75.01540899923914
CISD energy: -75.0147705755639
FCI energy:  -75.01553018949102


Number of qubits: 14


In [3]:
molecule.orb_symm_labels

array(['A1', 'A1', 'B1', 'A1', 'B2', 'A1', 'B1'], dtype='<U2')

In [4]:
molecule.groupname

'C2v'

In [5]:
molecule.topgroup

'C2v'

http://gernot-katzers-spice-pages.com/character_tables/C2v.html 

- $C_{2v}$ point group is generated by $C_{2}$ and $\sigma_{h}$

See point group at:

http://symmetry.jacobs-university.de/cgi-bin/group.cgi?group=402&option=4

- Z2 symmetry needs value in table to be +1 or -1 !!! (cannot be other vals)
- also point group considered must be Abelian

In [10]:
# C2 : A1 = +1, A2=+1, B1=-1, B2=-1

#𝜎_ℎ: A1 = +1, A2=-1, B1=+1, B2=-1

# need to look at sym labels of occupied orbitals

In [173]:
# note can only have values of +1 or -1 here!
C2 = {'A1': 1,
      'A2': 1,
      'B1':-1,
      'B2':-1,}

sig_h = { 'A1': 1,
          'A2':-1,
          'B1': 1,
          'B2':-1,}

In [174]:
# rks_double_occ = molecule.hf_array[::2] + molecule.hf_array[1::2]

labels_alpha = molecule.orb_symm_labels[np.where(molecule.hf_array[::2])]
labels_beta = molecule.orb_symm_labels[np.where(molecule.hf_array[1::2])]

In [175]:
labels = np.dstack((labels_alpha,labels_beta)).flatten()
labels

array(['A1', 'A1', 'A1', 'A1', 'B1', 'B1', 'A1', 'A1', 'B2', 'B2'],
      dtype='<U2')

In [176]:
# Look at C2 first
# only get a -1 sign for B1 and B2 

#['A1', 'A1', 'A1', 'A1', 'B1',    'B1',   'A1', 'A1', 'B2',     'B2']
#  I   ⊗ I  ⊗   I  ⊗ I  ⊗(aa-2I) ⊗ (aa-2I)⊗  I ⊗  I  ⊗ (aa-2I) ⊗(aa-2I)

In [252]:
ferms = []
symm_dict = sig_h # [C2 or sig_h]
for qind, term in enumerate(labels):
    if symm_dict[term]==-1:
        # I - 2Ni
        
        op = FermionOperator(f'',1) + FermionOperator(f'{qind}^ {qind}', -2)
        ferms.append(op)
        
#         ferms.append(FermionOperator(f'',1))
#         ferms.append(FermionOperator(f'{qind}^ {qind}',-2))
        



In [253]:
ferms

[1 [] +
 -2.0 [8^ 8],
 1 [] +
 -2.0 [9^ 9]]

In [258]:
from functools import reduce
from openfermion import jordan_wigner


# C2_sym_op_ferm = reduce(lambda x,y:x*y, ferms)
C2_sym_op_ferm = reduce(lambda x,y:x+y, ferms)

jordan_wigner(C2_sym_op_ferm)

(1+0j) [Z8] +
(1+0j) [Z9]

In [260]:
[jordan_wigner(f) for f in ferms]

[(1+0j) [Z8], (1+0j) [Z9]]

In [261]:
# string_list = [f'{qind} {qind}^' for qind, term in enumerate(labels) if term[0]=='B']

# sym = FermionOperator(' '.join(string_list),1)

In [262]:
# [jordan_wigner(f) for f in ferms]

In [263]:
from symmer.symplectic import StabilizerOp

stabs = StabilizerOp.symmetry_basis(molecule.H_q, commuting_override=True)
stabs

 1 IIIIIIIIZZIIII 
 1 ZIZIZIZIIZZIZI 
 1 IZIZIZIZIZIZIZ

In [264]:
stabs[0] * stabs[1] * stabs[2]

 1.000+0.000j ZZZZZZZZZZZZZZ

In [265]:
stabs[0] * stabs[2]

 1.000+0.000j IZIZIZIZZIIZIZ